In [80]:
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
MEA = '/102016_MEAC'

pathtimestamps = '/Users/Tolkien/Box Sync/Mazuski_VIPChR2OpticalTagging_MEADATA'+MEA+'_sorting_results/goodneurons/' # change the path depending on MEA

timestamps = os.listdir(pathtimestamps) # lists the directory containing all the timestamps from that MEA
timestamps = np.sort(timestamps) # so that folder ends up at the end of the list

pathtimestamps2 = '/Users/Tolkien/Documents/Mazuski_MEAresults/JTKcycle_periodandphase'+MEA+'.csv'

pathtosave = '/Users/Tolkien/Documents/Mazuski_MEAresults'+MEA

#os.mkdir(pathtosave)

In [81]:
#Calculate Maximums and Print out max and graphs

x = 0

for i in range(len(timestamps)-1):
    
    if timestamps[i] == '.DS_Store':
        x = 10000
        continue
    else:
        pathfile = pathtimestamps + timestamps[i]
        neuron = np.genfromtxt(pathfile, delimiter = ',') #imports specified neuron
        neuron = np.sort(neuron)

        periodphase = np.genfromtxt(pathtimestamps2, delimiter = ',') #imports period/phase values for entire MEA
        # starts at index 1 - period is i,0 JTK cycle phase is i,1 Both are in h

        # plan is to use JTK phase marker and then subtract half of the JTK period estimate
        # if that # is negative it means that the cycle is not intact - cannot calculate values
        # start with next cycle - in other words phase marker plus half the JTK period estimate

        counts, bins = np.histogram(neuron, int(neuron[-1]/600)) # calculates histogram in 10min bins
        counts = np.float_(counts)/600
        if x == 10000:
            jtkphase = periodphase[i,1]*6
            jtkperiod = periodphase[i,0]*6
        else:    
            jtkphase = periodphase[i+1,1]*6
            jtkperiod = periodphase[i+1,0]*6

        startpoint = jtkphase - (jtkperiod/2)
        if np.isnan(jtkphase):
            print 'ERROR ' + timestamps[i]
            continue

        if len(counts) <= jtkperiod:
            print 'ERROR '+timestamps[i]
            continue

        if startpoint > 0:
            startpoint = startpoint
        else:
            startpoint = jtkphase + (jtkperiod/2)


        day1start = startpoint
        day2start = startpoint + jtkperiod
        day3start = day2start + jtkperiod


        day1max = max(counts[day1start:day2start])
        day1maxtime = np.argmax(counts[day1start:day2start])
        day1maxtime = day1maxtime + day1start

        startISI = day1maxtime*10*60 - 1800
        endISI = day1maxtime*10*60 + 1800

        starttimestamps = day1maxtime*10*60 - 100
        endtimestamps = day1maxtime*10*60 +100
        timestamp = []

        for j in range(len(neuron)-1):
            if neuron[j] > starttimestamps and neuron[j] < endtimestamps:
                timestamp.append(neuron[j]-starttimestamps)
        np.savetxt(pathtosave+'/timestamps'+timestamps[i]+'.csv', timestamp, delimiter = ',')
        ISI = []

        for j in range(len(neuron)-1):
            if neuron[j] > startISI and neuron[j] < endISI:
                ISI.append(neuron[j+1]-neuron[j])

        ISIhist, bins = np.histogram(ISI, 100, range=(0,1))

        np.savetxt(pathtosave+'/ISI'+timestamps[i]+'.csv', ISIhist, delimiter = ',')
        
        if day3start < len(counts):
            day2max = max(counts[day2start:day3start])
            day2maxtime = np.argmax(counts[day2start:day3start])
            day2maxtime = day2maxtime + day2start
            
            startISI2 = day2maxtime*10*60 - 1800
            endISI2 = day2maxtime*10*60 + 1800
            
            starttimestamps2 = day2maxtime*10*60 - 100
            endtimestamps2 = day1maxtime*10*60 + 100
            timestamp2 = []
            
            for k in range(len(neuron)-1):
                if neuron[k] > starttimestamps2 and neuron[k] < endtimestamps:
                    timestamp2.append(neuron[k]-starttimestamps)
            np.savetxt(pathtosave+'/timestamps2'+timestamps[i]+'.csv', timestamp2, delimiter = ',')
            ISI2 = []
            
            for k in range(len(neuron)-1):
                if neuron[k] > startISI2 and neuron[k] < endISI2:
                    ISI2.append(neuron[k+1]-neuron[k])
                    
            ISIhist2, bins2 = np.histogram(ISI2, 100, range=(0,1))
            
            np.savetxt(pathtosave+'/ISI2'+timestamps[i]+'.csv', ISIhist2, delimiter = ',')
            
print 'finished!'
        

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:52: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:77: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:78: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


finished!
